In [3]:
!pip install tensorflow-addons
!pip install -U tensorboard-plugin-profile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 4.3 MB/s 


In [5]:
# google drive
from google.colab import drive
drive.mount('/content/drive')

from tqdm import tqdm
import os
import yaml
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' ## hide tf warnings

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
from keras.engine.base_layer import Layer
from tensorflow.keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects

from drive.MyDrive.Kaggle.Imputation_062022.Previous_work.src.functions import *
from drive.MyDrive.Kaggle.Imputation_062022.Previous_work.models.MLP_residuals_v2 import *


class Mish(Activation):
    '''
    Mish Activation Function.
    .. math::
        mish(x) = x * tanh(softplus(x)) = x * tanh(ln(1 + e^{x}))
    Shape:
        - Input: Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
        - Output: Same shape as the input.
    Examples:
        >>> X = Activation('Mish', name="conv1_act")(X_input)
    '''

    def __init__(self, activation, **kwargs):
        super(Mish, self).__init__(activation, **kwargs)
        self.__name__ = 'Mish'


def mish(x):
    return x*K.tanh(K.softplus(x))

get_custom_objects().update({'Mish': Mish(mish)})

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: # detect GPUs
    #strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)

Number of accelerators:  1


In [ ]:
pd.read_csv('/content/drive/MyDrive/Kaggle/Imputation_062022/F4_transformed.csv')

,Unnamed: 0,F_4_0,F_4_1,F_4_2,F_4_3,F_4_4,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
0,0,6.033415,1.059504,-0.133129,-0.099809,-0.649547,3.669890,0.782401,0.266030,-0.509082,0.200776,0.396275,5.744312,3.846432,1.221378,1.436190
1,1,-1.641235,-1.197116,-0.544232,-1.287375,1.329688,-2.945907,-0.749004,2.406945,-0.060004,0.780156,-0.260240,4.875188,-4.175343,-2.784360,-0.062922
2,2,1.999596,3.812096,NaN,0.373527,0.449468,2.220437,0.467883,2.540733,0.377879,-0.253496,-0.000833,-0.116852,-2.117650,3.568032,-0.128055
3,3,-2.501545,0.544501,0.907678,-0.787480,2.803308,1.977976,-4.918797,-0.839800,-0.251243,0.708988,-0.513518,7.761853,-4.687275,-3.126407,0.542717
4,4,-0.228658,-0.460594,1.197291,-0.693992,NaN,0.969331,2.467455,3.459185,0.977849,0.744767,0.602891,-6.457764,-2.846537,3.636661,-0.637479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,999995,1.182599,-5.229601,0.778391,-0.190961,2.746968,1.777425,-0.303337,5.508448,-0.847600,1.429258,-0.748570,-2.376250,1.773245,5.156470,-0.218520
999996,999996,-4.614906,-1.652754,0.904842,0.344790,2.499703,1.885172,-1.329129,-0.071648,-0.157065,0.073103,-0.561813,-1.301552,0.190552,-0.345558,-0.324399
999997,999997,1.097715,0.293081,-0.826888,-0.281589,1.242981,2.842904,2.979690,-3.387820,NaN,0.098060,0.620598,3.409637,-0.582692,-1.513446,-0.847593
999998,999998,-0.843665,3.742854,-0.186917,-0.793919,-2.484428,-1.145052,-5.422207,-3.803963,-1.276403,0.970700,0.402990,0.829258,-1.081178,3.050068,0.004836


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Kaggle/Imputation_062022/data_transformed.csv', index_col='row_id')
# F4data = pd.read_csv('/content/drive/MyDrive/Kaggle/Imputation_062022/F4_transformed.csv', index_col='row_id')
sample = pd.read_csv('/content/drive/MyDrive/Kaggle/Imputation_062022/src/data/sample_submission.csv', index_col='row-col')

In [ ]:
F4data = data[F4]

In [ ]:
with open('/content/drive/MyDrive/Kaggle/Imputation_062022/src/config.YAML', 'r') as f:
    config = yaml.load(f)

In [ ]:
set_seed(config['SEED'])
col_list, F1, F2, F3, F4, missing_cols = get_lists(data)

In [ ]:
def high_correlated(col, n_columns):
    return data.corrwith(data[col]).abs().sort_values(ascending=False)[1:n_columns].index.to_list() #+ ['n_missing']

In [ ]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
# !rm -r logs/fit
%reload_ext tensorboard
%tensorboard --logdir logs/fit

In [ ]:
# %%time
data_imputed = pd.DataFrame() 
loss_per_feature={}
history_feats = {}

# prescale and impute data to speed up training
scaler = StandardScaler()
si = SimpleImputer()
new_df = si.fit_transform(F4data[F4].copy())
new_df = pd.DataFrame(scaler.fit_transform(new_df), columns=F4, index=F4data.index)

# Iterate over the columns that contain Nan values
for idx, col in tqdm(enumerate(F4)):
    print(col)
    predictions=[]
    validation_loss=[]
    training_loss = []
    history_per = {}

    # Mask to access not_null part of the current column
    not_null = ~F4data[col].isnull()
    
    # Train dataset (includes non-null part of current column)
    train = new_df.loc[not_null, :].drop(columns=col)
    
    # Test dataset (includes null part of current column)
    test = new_df.loc[~not_null, :].drop(columns=col)

    # Split y column out
    y = data[col].loc[not_null]

    # Cross validation type
    kf = KFold(n_splits = config['N_SPLITS'])
    
    # Splitting data to train and validation
    for fold, (train_idx, val_idx) in enumerate(kf.split(train)):

        X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx,]
        X_test = test
               

        # Running NN model 
        with strategy.scope():
            model = nn_model(X_train, activation='Mish', lr=config['INITIAL_LR'], dropout_pct=config['DROPOUT_PCT'])

        history = model.fit(X_train, y_train, 
                            epochs= config['MAX_EPOCHS'],
                            callbacks=callbacks(config['LR_FACTOR'], config['LR_PATIENCE'], config['ES_PATIENCE']),
                            validation_data=(X_val, y_val),
                            shuffle=True,
                            verbose=1,
                            batch_size=config['BATCH_SIZE']
        )
        
        # Make an out-of-fold prediction
        y_preds = model.predict(X_test)
        
        # Add y_preds to a list
        predictions.append(y_preds)
        
        # Save loss for current fold
        validation_loss.append(history.history["val_loss"][-1])
        training_loss.append(history.history['loss'][-1])

        # Save history for modelling
        history_per[str(fold)] = history.history

    # Save history for modelling
    history_feats[str(col)] = [item for item in history_per.items()]

    # Caluculate the mean of oof predictions
    mean_values = np.array(predictions).mean(axis = 0)
    
    # Save mean-loss for current feature
    loss_per_feature[col] = np.mean(validation_loss)
    
    # Specifying column to impute
    imputed_feature = data[col].copy()
    
    # Filling missing values
    imputed_feature.loc[~not_null] =  mean_values.ravel()
    
    # Concatenate imputed columns
    data_imputed = pd.concat([data_imputed, imputed_feature],axis = 1)

# Replace columns with imputed columns 
data[F4] = data_imputed[F4]

0it [00:00, ?it/s]

F_4_0
Epoch 1/100
54/54 [==============================] - 4s 25ms/step - loss: 1.7421 - rmse: 1.7374 - val_loss: 2.4311 - val_rmse: 2.4307 - lr: 0.0200
Epoch 2/100
54/54 [==============================] - 1s 18ms/step - loss: 1.2317 - rmse: 1.2305 - val_loss: 1.4717 - val_rmse: 1.4718 - lr: 0.0200
Epoch 3/100
54/54 [==============================] - 1s 18ms/step - loss: 1.0237 - rmse: 1.0226 - val_loss: 1.0925 - val_rmse: 1.0928 - lr: 0.0200
Epoch 4/100
54/54 [==============================] - 1s 17ms/step - loss: 0.9099 - rmse: 0.9092 - val_loss: 1.0532 - val_rmse: 1.0536 - lr: 0.0200
Epoch 5/100
54/54 [==============================] - 1s 17ms/step - loss: 0.8553 - rmse: 0.8550 - val_loss: 0.9982 - val_rmse: 0.9985 - lr: 0.0200
Epoch 6/100
54/54 [==============================] - 1s 18ms/step - loss: 0.8229 - rmse: 0.8225 - val_loss: 0.9774 - val_rmse: 0.9777 - lr: 0.0200
Epoch 7/100
54/54 [==============================] - 1s 18ms/step - loss: 0.8030 - rmse: 0.8029 - val_loss: 0.90

1it [04:05, 245.91s/it]

F_4_1
Epoch 1/100
54/54 [==============================] - 5s 33ms/step - loss: 1.4095 - rmse: 1.4047 - val_loss: 2.2645 - val_rmse: 2.2642 - lr: 0.0200
Epoch 2/100
54/54 [==============================] - 1s 20ms/step - loss: 1.0140 - rmse: 1.0126 - val_loss: 1.3423 - val_rmse: 1.3422 - lr: 0.0200
Epoch 3/100
54/54 [==============================] - 1s 18ms/step - loss: 0.9342 - rmse: 0.9334 - val_loss: 1.0319 - val_rmse: 1.0319 - lr: 0.0200
Epoch 4/100
54/54 [==============================] - 1s 19ms/step - loss: 0.8873 - rmse: 0.8872 - val_loss: 0.9481 - val_rmse: 0.9481 - lr: 0.0200
Epoch 5/100
54/54 [==============================] - 1s 19ms/step - loss: 0.8514 - rmse: 0.8510 - val_loss: 0.8866 - val_rmse: 0.8867 - lr: 0.0200
Epoch 6/100
54/54 [==============================] - 1s 19ms/step - loss: 0.8215 - rmse: 0.8217 - val_loss: 0.8801 - val_rmse: 0.8801 - lr: 0.0200
Epoch 7/100
54/54 [==============================] - 1s 18ms/step - loss: 0.7986 - rmse: 0.7981 - val_loss: 0.85

2it [08:24, 253.65s/it]

...

13it [1:01:56, 299.51s/it]

F_4_13
Epoch 1/100
54/54 [==============================] - 4s 26ms/step - loss: 1.1237 - rmse: 1.1190 - val_loss: 2.2224 - val_rmse: 2.2220 - lr: 0.0200
Epoch 2/100
54/54 [==============================] - 1s 19ms/step - loss: 0.7586 - rmse: 0.7582 - val_loss: 1.2443 - val_rmse: 1.2439 - lr: 0.0200
Epoch 3/100
54/54 [==============================] - 1s 19ms/step - loss: 0.7137 - rmse: 0.7140 - val_loss: 0.7613 - val_rmse: 0.7609 - lr: 0.0200
Epoch 4/100
54/54 [==============================] - 1s 20ms/step - loss: 0.6887 - rmse: 0.6889 - val_loss: 0.6829 - val_rmse: 0.6826 - lr: 0.0200
Epoch 5/100
54/54 [==============================] - 1s 19ms/step - loss: 0.6640 - rmse: 0.6638 - val_loss: 0.7516 - val_rmse: 0.7513 - lr: 0.0200
Epoch 6/100
54/54 [==============================] - 1s 18ms/step - loss: 0.6544 - rmse: 0.6541 - val_loss: 0.6617 - val_rmse: 0.6614 - lr: 0.0200
Epoch 7/100
54/54 [==============================] - 1s 18ms/step - loss: 0.6246 - rmse: 0.6246 - val_loss: 0.7

14it [1:06:12, 286.34s/it]

F_4_14
Epoch 1/100
54/54 [==============================] - 4s 32ms/step - loss: 0.5746 - rmse: 0.5707 - val_loss: 0.6867 - val_rmse: 0.6867 - lr: 0.0200
Epoch 2/100
54/54 [==============================] - 1s 20ms/step - loss: 0.2837 - rmse: 0.2835 - val_loss: 0.4552 - val_rmse: 0.4551 - lr: 0.0200
Epoch 3/100
54/54 [==============================] - 1s 20ms/step - loss: 0.2614 - rmse: 0.2613 - val_loss: 0.2942 - val_rmse: 0.2942 - lr: 0.0200
Epoch 4/100
54/54 [==============================] - 1s 18ms/step - loss: 0.2477 - rmse: 0.2475 - val_loss: 0.2463 - val_rmse: 0.2463 - lr: 0.0200
Epoch 5/100
54/54 [==============================] - 1s 20ms/step - loss: 0.2373 - rmse: 0.2372 - val_loss: 0.2398 - val_rmse: 0.2399 - lr: 0.0200
Epoch 6/100
54/54 [==============================] - 1s 18ms/step - loss: 0.2277 - rmse: 0.2277 - val_loss: 0.2328 - val_rmse: 0.2329 - lr: 0.0200
Epoch 7/100
54/54 [==============================] - 1s 18ms/step - loss: 0.2199 - rmse: 0.2197 - val_loss: 0.2

15it [1:11:27, 285.82s/it]


In [ ]:
loss_df=pd.DataFrame(loss_per_feature,index=['Validation_RMSE']).T.sort_values(by='Validation_RMSE',
                                                                               ascending=False)

loss_df

,Validation_RMSE
F_4_6,0.685783
F_4_1,0.667109
F_4_7,0.666841
F_4_11,0.654956
F_4_0,0.653394
F_4_12,0.634303
F_4_5,0.582223
F_4_4,0.562274
F_4_13,0.496952
F_4_3,0.219743


# Model Comparison/insights
- Training time was down to 3 hours, compared to 12+ hours from MLP1, the second version was not much faster either. So, this model is a massive improvment time-wise (this was run with 3 folds, MLP1 used 5, but the difference is still considerable)
- Results: It achieved an RMSE 0.89029 at batch size ~12000 or so, compared to ~0.863 at 1024. The was no substantial change when stepping down to 1024
- loss and val loss are very close together which indicates the model is at least not overfitting (loss would be smaller than val loss). Since it is not overfitting, and I far surpassed my decreased training times, I will increase model complexity.

In [ ]:
si = SimpleImputer(strategy='mean')
trans = pd.DataFrame(si.fit_transform(data), columns=data.columns)
sub = make_submission(trans, sample)

sub.to_csv("drive/MyDrive/Kaggle/Imputation_062022/submissions/MLP_residuals_v3_data_ptransformed.csv")

In [ ]:
loss_df.to_csv('drive/MyDrive/Kaggle/Imputation_062022/src/val_loss_MLP_residuals_v3_data_ptransformed.csv')

In [ ]:
with open('drive/MyDrive/Kaggle/Imputation_062022/src/MLP_residuals_v3_data_ptransformed_history.pkl', 'wb') as f:
    pickle.dump(history_feats, f)